# Tutorial 5: Stacking Sats Challenge

### Strategy Submission Template

**Hosted on [Hypertrial.ai](https://www.hypertrial.ai/)**

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hypertrial/stacking_sats_challenge/blob/main/tutorials/5.%20Strategy%20Submission%20Template.ipynb)

Welcome to the submission template for the Stacking Sats Challenge.

This notebook replicates the structure used by the evaluation engine to test all participant strategies. If your strategy passed all the tests in [Tutorial 4 Strategy Development Template](https://github.com/hypertrial/stacking_sats_challenge/blob/main/tutorials/4.%20Strategy%20Development%20Template.ipynb), it could still fail the tests here due to enviroment differences. Which is why we strongly encourage participants to use a virtual enviroment while developing and testing their strategies (complete [Tutorial 3. Virtual Env Setup](https://github.com/hypertrial/stacking_sats_challenge/blob/main/tutorials/3.%20Virtual%20Env%20Setup.ipynb) if you haven't already).   

---

## 🛠️ What You’re Expected to Do

- Paste your strategy code in the multi-line string named `model_code` in cell 3
- Run the entire notebook and get "✅ Strategy is ready for submission."
- Submit the exact same strategy code function for evaluation on [Hypertrial.ai](https://www.hypertrial.ai/)

---

> ⚠️ Do not change function names, decorators, or global config values unless explicitly allowed.  
> Your entry must adhere to this template to be considered valid.


In [2]:
!pip install --upgrade --force-reinstall hypertrial

  Using cached hypertrial-0.1.14-py3-none-any.whl.metadata (5.5 kB)
  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached coinmetrics_api_client-2025.4.15.13-py3-none-any.whl.metadata (3.4 kB)
  Using cached pytest-8.3.5-py3-none-any.whl.metadata (7.6 kB)
  Using cached pandas_datareader-0.10.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached scipy-1.15.2-cp310-cp310-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached psutil-7.0.0-cp36-abi3-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached bandit-1.8.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached safety-3.3.1-py3-none-any.whl.metadata (11 kB)
  Using cached pylint-3.3.6-py3-none-any.whl.metadata (12 kB)
  Using cached pytest_cov-6.1.1-py3-none-any.whl.metadata (28 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.

## 🔍 Overview

In this tutorial, we provide a helper function to run your strategy code from within the notebook.  
You will see:
- A function that saves your strategy to a temporary `.py` file.
- The execution of the strategy using the Hypertrial CLI.
- An optional cleanup step to remove the temporary file after execution.


In [4]:
def run_strategy_from_notebook(strategy_code: str, filename: str = "temp_strategy.py", cleanup: bool = False):
    """
    Save strategy code to a .py file and run it with core.main using --strategy-file.
    
    Args:
        strategy_code (str): The full strategy code as a string
        filename (str): The .py filename to save
        cleanup (bool): If True, deletes the file after running
    """
    import os

    # Save code to .py file
    with open(filename, "w") as f:
        f.write(strategy_code)
    print(f"✅ Saved strategy to {filename}")

    # Run the strategy using Hypertrial CLI
    print("🚀 Running strategy...")
    exit_code = os.system(f"python3 -m core.main --strategy-file {filename} --standalone")

    # Optional cleanup
    if cleanup:
        os.remove(filename)
        print(f"🧹 Deleted {filename} after execution.")

    # Return exit code in case you want to handle it
    return exit_code

## 🔧 Next Steps

1. **Insert Your Strategy:**  
   Paste your strategy code in the multi-line string named `model_code`.
   
2. **Validate Your Submission:**  
   Run the entire notebook to ensure that your strategy is tested correctly.
   
3. **Submission Guidelines:**  
   If you see "✅ Strategy passed all validation checks." that means you are ready to submit your code on Hypertrial.
   
4. **Troubleshooting:**  
   If you encounter any issues during execution, verify that all required dependencies are installed and that your Python environment is correctly configured. Or it may mean that your model's logic does not meet the criteria. We suggest going back to tutorial 3 and running your own set of tests to ensure the model is behaving as expected. 

In [16]:
model_code = '''
import pandas as pd
import numpy as np
from typing import Dict, Any
from core.config import BACKTEST_START, BACKTEST_END, MIN_WEIGHT
from core.strategies import register_strategy

def construct_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Construct technical indicators used for the strategy.
    Uses only past data for calculations to avoid look-ahead bias.
    
    Args:
        df: DataFrame with price data
        
    Returns:
        DataFrame with added technical indicators
    """
    df = df.copy()
    # Shift the btc_close column by one to use only past data for our calculations
    past_close = df['btc_close'].shift(1)
    # Calculate 200-day moving average
    df['ma200'] = past_close.rolling(window=200, min_periods=1).mean()
    # Calculate 200-day standard deviation
    df['std200'] = past_close.rolling(window=200, min_periods=1).std()
    return df

# Example Ethereum wallet address - replace with real one for actual submissions
ETH_WALLET_ADDRESS = "0x71C7656EC7ab88b098defB751B7401B5f6d8976F"

@register_strategy(ETH_WALLET_ADDRESS)
def compute_weights(df: pd.DataFrame) -> pd.Series:
    """
    Computes daily DCA weights with a 200-day moving average strategy.
    Increases weight on days when price is below MA, redistributing from future days.
    
    Strategy logic:
    1. Start with uniform weights across each market cycle
    2. For days when price < 200MA, boost weight proportional to distance below MA
    3. Redistribute the excess weight from future days within a rebalance window
    4. Maintain minimum weight constraints for all days
    
    Args:
        df: DataFrame with BTC price data
        
    Returns:
        Series of daily investment weights, summing to 1.0 per market cycle
    """
    # Strategy parameters
    REBALANCE_WINDOW = 365 * 2  # Redistribute weight from up to 2 years ahead
    ALPHA = 1.25  # Multiplier for how much to boost weight based on z-score
    
    df_work = df.copy()
    df_work = construct_features(df)
    
    # Filter to backtest period only
    df_backtest = df_work.loc[BACKTEST_START:BACKTEST_END]
    weights = pd.Series(index=df_backtest.index, dtype=float)
    
    # Group by 4-year market cycles
    start_year = pd.to_datetime(BACKTEST_START).year
    cycle_labels = df_backtest.index.to_series().apply(lambda dt: (dt.year - start_year) // 4)
    
    # Process each market cycle separately to maintain weight sum = 1.0 per cycle
    for cycle, group in df_backtest.groupby(cycle_labels):
        N = len(group)
        base_weight = 1.0 / N  # Start with uniform weight distribution
        temp_weights = np.full(N, base_weight)
        strategy_active = True  # Flag to stop adjustments if constraints can't be met
        
        # Process each day in the cycle
        for i in range(N):
            if not strategy_active:
                break
            
            price = group['btc_close'].iloc[i]
            ma200 = group['ma200'].iloc[i]
            std200 = group['std200'].iloc[i]
            
            # Skip days with insufficient history
            if pd.isna(ma200) or pd.isna(std200) or std200 <= 0:
                continue
            
            # Apply weight boost when price is below MA
            if price < ma200:
                # Calculate z-score (standard deviations below MA)
                z = (ma200 - price) / std200
                boost_multiplier = 1 + ALPHA * z
                current_weight = temp_weights[i]
                boosted_weight = current_weight * boost_multiplier
                excess = boosted_weight - current_weight
                
                # Determine which future days to redistribute from
                start_redistribution = max(N - REBALANCE_WINDOW, i + 1)
                if start_redistribution >= N:
                    continue  # No future days to redistribute from
                
                redistribution_idx = np.arange(start_redistribution, N)
                if len(redistribution_idx) == 0:
                    continue
                    
                # Calculate reduction per future day
                reduction = excess / len(redistribution_idx)
                projected = temp_weights[redistribution_idx] - reduction
                
                # Only apply changes if minimum weight constraint is satisfied
                if np.all(projected >= MIN_WEIGHT):
                    temp_weights[i] = boosted_weight
                    temp_weights[redistribution_idx] -= reduction
                else:
                    # Stop strategy adjustments if constraints can't be met
                    strategy_active = False
        
        # Assign weights back to the original index
        weights.loc[group.index] = temp_weights
    
    return weights
'''

# Run it
run_strategy_from_notebook(model_code, filename="dynamic_dca_200ma.py", cleanup=False)

✅ Saved strategy to dynamic_dca_200ma.py
🚀 Running strategy...


/Users/mashkani/miniconda3/envs/stacking-sats-env/lib/python3.10/runpy.py:126: RuntimeWarning: 'core.main' found in sys.modules after import of package 'core', but prior to execution of 'core.main'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
INFO:root:Loading BTC data from core/data/btc_price_data.csv
INFO:root:Loaded 5381 records from 2010-07-18 00:00:00 to 2025-04-10 00:00:00
INFO:core.security.utils:Running Bandit security analysis on /Users/mashkani/Documents/hypertrial/Stacking Sats Challenge/Tutorials/dynamic_dca_200ma.py
INFO:core.security.utils:Bandit security scan: 0 issues found (High: 0, Medium: 0, Low: 0)
INFO:core.security.utils:Strategy file /Users/mashkani/Documents/hypertrial/Stacking Sats Challenge/Tutorials/dynamic_dca_200ma.py passed security validation
INFO:core.strategy_loader:Loading strategy from file: /Users/mashkani/Documents/hypertrial/Stacking Sats Challenge/Tutorials/dynamic_dca_200ma.py
INFO:core.strategies:Registered strategy: 0

Weight sums by cycle (should be close to 1.0):
Cycle 2013–2016: 1.0000
Cycle 2017–2020: 1.0000
Cycle 2021–2024: 1.0000

SPD Metrics for 0x71C7656EC7ab88b098defB751B7401B5f6d8976F:
Dynamic SPD:
  min: 3485.52
  max: 596340.41
  mean: 208274.07
  median: 24996.29

✅ Strategy passed all validation checks.


INFO:core.spd:Computing SPD metrics for '0x71C7656EC7ab88b098defB751B7401B5f6d8976F' from 2013-01-01 to 2024-12-31


0